# Geoinformatics Project
---
# LEO satellites orbits estimation
###### Angelica Iseni, Emma Lodetti

The **goal** of this project is to compute the position of each LEO (Low Earth orbit) satellite in the constellation for every second of the day.

In this script you will be asked to enter: 
- The **number of orbital planes** of the costellation
- The **number of satellites** per orbital plane
- The **inclination** of the orbital planes with respect to the reference equatorial plane

This information will be used to create almanacs of the constellation satellites.

In [1]:
# Import libraries
import ipywidgets as widgets
from IPython.display import display
from IPython.display import FileLink
import os

### Input parameters
Insert number of orbits, number of satellites per orbit and inclination of the orbit.

In [2]:
# Widget creation
num_orbits = widgets.IntText(description='Number of orbits:', value=0)
num_satellites = widgets.IntText(description='Number of satellites per orbit:', value=0)
orbit_inclination = widgets.FloatSlider(description='Orbit inclination:', min=0, max=90, step=1)

def user_input(var):
    n_orbits = num_orbits.value
    n_satellites = num_satellites.value
    inclination = orbit_inclination.value
    if n_orbits < 0 or n_satellites < 0 or inclination < 0:
        print('Error: The numbers cannot be negative. Please insert valid numbers.')
    else:
        print(f'You have selected {n_orbits} orbits with {n_satellites} satellites each, with an inclination of {inclination} degrees.')
    
# Submit button
submit_button = widgets.Button(description='Submit')
submit_button.on_click(user_input)

display(num_orbits, num_satellites, orbit_inclination, submit_button)

IntText(value=0, description='Number of orbits:')

IntText(value=0, description='Number of satellites per orbit:')

FloatSlider(value=0.0, description='Orbit inclination:', max=90.0, step=1.0)

Button(description='Submit', style=ButtonStyle())

You have selected 4 orbits with 3 satellites each, with an inclination of 40.0 degrees.


### Almanacs creation as txt files

In [ ]:
# Orbital radius definition
orbit_radius = 8500

def txt_creation(var):
    n_orbits = num_orbits.value
    n_satellites = num_satellites.value
    inclination = orbit_inclination.value
    
    # txt content
    output_content = f'Number of orbits: {n_orbits}\nNumber of satellites per orbit: {n_satellites}\nOrbit inclination: {inclination} degrees\nRadius: {orbit_radius} km'
    
    # Insert content in output file
    with open('orbital_data.txt', 'w') as file:
        file.write(output_content)
    
    print('"orbital_data.txt" file successfully created!')
    
    # Creare un link per scaricare il file
    display(FileLink('orbital_data.txt'))

# Submit button
submit_button = widgets.Button(description='Create file')
submit_button.on_click(txt_creation)

display(submit_button)

# TODO: download button
# TODO: non sovrascrivere
# aggiungere GMe e OmegaEDot

In [ ]:
# Orbital radius definition
orbit_radius = 8500

num_orbits = int(num_orbits.value)
num_satellites = int(num_satellites.value)
inclination = int(orbit_inclination.value)

for i_orb in range(1, num_orbits + 1):
    Omega0 = (i_orb - 1) * 180 / num_orbits
    
    # Ciclo interno sui satelliti
    for i_sat in range(1, num_satellites + 1):
        M0 = (360 / num_satellites) * (i_sat - 1) + 360 / num_satellites * ((i_orb - 1) / num_orbits)
        
        print(f'Omega0: {Omega0}, M0: {M0} per orbita {i_orb}, satellite {i_sat}')

In [3]:
num_orbits = int(num_orbits.value)
num_satellites = int(num_satellites.value)
inclination = int(orbit_inclination.value)

# Almanac folder creation
if not os.path.exists('Almanacs'):
    os.makedirs('Almanacs')    

# Orbital radius definition
orbit_radius = 8500

def create_satellite_txt(i_orbit, i_satellite):
    # Nome del satellite
    satellite_name = f'LEO {i_orbit:02}{i_satellite:02}'
    
    # M0 computation
    M0 = 360 / num_satellites * (i_satellite - 1) + 360 / num_satellites * ((i_orbit - 1) / num_orbits)
    
    # Omega0 computation
    Omega0 = (i_orbit - 1) * 180 / num_orbits
    
    # Contenuto del file di testo
    output_content = f'Satellite Name: {satellite_name}\nRadius: {orbit_radius} km\nM0: {M0}\nOmega0: {Omega0}\nInclination: {int(orbit_inclination.value)} degrees'
    
    # Percorso del file di testo
    file_path = f'Almanacs/{satellite_name}.txt'
    
    # Scrivere il contenuto nel file di output
    with open(file_path, 'w') as file:
        file.write(output_content)

# Ciclo esterno sui piani orbitali
for i_orb in range(1, num_orbits + 1):
    # Ciclo interno sui satelliti
    for i_sat in range(1, num_satellites + 1):
        create_satellite_txt(i_orb, i_sat)

In [ ]:
num_orbits = int(num_orbits.value)
num_satellites = int(num_satellites.value)
inclination = int(orbit_inclination.value)

# Creazione della cartella "Almanacs" se non esiste
if not os.path.exists('Almanacs'):
    os.makedirs('Almanacs')    

# Orbital radius definition
orbit_radius = 8500

def create_satellite_txt(orbit_num, satellite_num):
    # Nome del satellite
    satellite_name = f'LEO {orbit_num:02}{satellite_num:02}'
    
    # Contenuto del file di testo
    output_content = f'Satellite Name: {satellite_name}\nRadius: {orbit_radius} km\nM0: {360 / num_satellites * (satellite_num - 1) + 360 / num_satellites * ((orbit_num - 1) / num_orbits)}\nOmega0: {(orbit_num - 1) * 180 / num_orbits}\nInclination: {int(orbit_inclination.value)} degrees'
    
    # Percorso del file di testo
    file_path = f'Almanacs/{satellite_name}.txt'
    
    # Scrivere il contenuto nel file di output
    with open(file_path, 'w') as file:
        file.write(output_content)

# Ciclo esterno sui piani orbitali
for i_orb in range(1, num_orbits + 1):
    # Ciclo interno sui satelliti
    for i_sat in range(1, num_satellites + 1):
        create_satellite_txt(i_orb, i_sat)